## GIF movie generator

Import packages and predictions from the pickle files

In [ ]:
import pickle
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from datetime import timedelta, datetime
from scipy.stats import johnsonsu
import pulp
from PIL import Image, ImageDraw

path = os.getcwdb().decode("utf-8")+"/data"

In [ ]:
scaler_x, X_scaled, y_a1, y_a2, y_sp, X_test_scaled, y_test_a1, y_test_a2, y_test_sp, feature_names, imp_order = pickle.load(open(path+"/pickle_1.pkl", "rb"))
auction1_predict_lstm, auction2_predict_lstm, system_price_predict_lstm = pickle.load(open(path+"/pickle_2_optuna_predict_lstm.pkl", "rb"))
auction1_predict_mlp, auction2_predict_mlp, system_price_predict_mlp = pickle.load(open(path+"/pickle_2_optuna_predict_alt.pkl", "rb"))
auction1_predict_qr, auction2_predict_qr, system_price_predict_qr, qt_a1, qt_a2, qt_sp = pickle.load(open(path+"/pickle_2_2.pkl", "rb"))
auction1_predict_sf, auction2_predict_sf, system_price_predict_sf = pickle.load(open(path+"/pickle_simple_fit.pkl", "rb"))


In [ ]:
auction1_predict_qr     = np.array(auction1_predict_qr)
auction2_predict_qr     = np.array(auction2_predict_qr)
system_price_predict_qr = np.array(system_price_predict_qr)

In [ ]:
x = np.arange(1,100)/100

In [ ]:
testing_start   = datetime(2022,3,2,0,0,0)
test_timestamps = [testing_start + timedelta(hours=i) for i in range(len(auction1_predict_lstm))]

Generate the pictures for the GIFs. Only first 720 data points used due to computational cost.

In [ ]:
for i in range(720):#range(len(auction1_predict_lstm)):
    y_a1_lstm = johnsonsu.ppf(x, *auction1_predict_lstm[i,:])
    y_a2_lstm = johnsonsu.ppf(x, *auction2_predict_lstm[i,:])
    y_sp_lstm = johnsonsu.ppf(x, *system_price_predict_lstm[i,:])
    y_a1_mlp  = johnsonsu.ppf(x, *auction1_predict_mlp[i+23,:])
    y_a2_mlp  = johnsonsu.ppf(x, *auction2_predict_mlp[i+23,:])
    y_sp_mlp  = johnsonsu.ppf(x, *system_price_predict_mlp[i+23,:])
    y_a1_qr   = johnsonsu.ppf(x, *auction1_predict_qr[i+23,:])
    y_a2_qr   = johnsonsu.ppf(x, *auction2_predict_qr[i+23,:])
    y_sp_qr   = johnsonsu.ppf(x, *system_price_predict_qr[i+23,:])
    y_a1_sf   = johnsonsu.ppf(x, *auction1_predict_sf[i+23,:])
    y_a2_sf   = johnsonsu.ppf(x, *auction2_predict_sf[i+23,:])
    y_sp_sf   = johnsonsu.ppf(x, *system_price_predict_sf[i+23,:])

    plt.clf()
    plt.ylabel("Predicted price")
    plt.xlabel("Quantile")
    plt.title("EPEX "+str(test_timestamps[i]))
    plt.ylim((0,800))
    plt.plot(x,y_a1_lstm)
    plt.plot(x,y_a1_mlp)
    plt.plot(x,y_a1_qr)
    plt.plot(x,y_a1_sf)
    plt.plot([0,1], [y_test_a1[i+23],y_test_a1[i+23]], color="black", lw=0.5)
    plt.savefig(path+"/quantile_gif_a1/quantile_pic_"+str(i)+".png", transparent=True, dpi=300, bbox_inches="tight")    
    plt.clf()
    plt.ylabel("Predicted price")
    plt.xlabel("Quantile")
    plt.title("Nord Pool "+str(test_timestamps[i]))
    plt.ylim((0,800))
    plt.plot(x,y_a2_lstm)
    plt.plot(x,y_a2_mlp)
    plt.plot(x,y_a2_qr)
    plt.plot(x,y_a2_sf)
    plt.plot([0,1], [y_test_a2[i+23],y_test_a2[i+23]], color="black", lw=0.5)
    plt.savefig(path+"/quantile_gif_a2/quantile_pic_"+str(i)+".png", transparent=True, dpi=300, bbox_inches="tight")      
    plt.clf()
    plt.ylabel("Predicted price")
    plt.xlabel("Quantile")
    plt.title("System Prices "+str(test_timestamps[i]))
    plt.ylim((0,800))
    plt.plot(x,y_sp_lstm)
    plt.plot(x,y_sp_mlp)
    plt.plot(x,y_sp_qr)
    plt.plot(x,y_sp_sf)
    plt.plot([0,1], [y_test_sp[i+23],y_test_sp[i+23]], color="black", lw=0.5)
    plt.savefig(path+"/quantile_gif_sp/quantile_pic_"+str(i)+".png", transparent=True, dpi=300, bbox_inches="tight")  

Generate GIFs out of the pictures for each time series.

In [ ]:
images = []
for i in range(720):
    images.append(Image.open(path+"/quantile_gif_a1/quantile_pic_"+str(i)+".png"))
images[0].save(path+"/movie_a1.gif", format='GIF', save_all=True, append_images=images[1:], duration=3, loop=0, transparency=0, disposal=2)
images = []
for i in range(720):
    images.append(Image.open(path+"/quantile_gif_a2/quantile_pic_"+str(i)+".png"))
images[0].save(path+"/movie_a2.gif", format='GIF', save_all=True, append_images=images[1:], duration=3, loop=0, transparency=0, disposal=2)
images = []
for i in range(720):
    images.append(Image.open(path+"/quantile_gif_sp/quantile_pic_"+str(i)+".png"))
images[0].save(path+"/movie_sp.gif", format='GIF', save_all=True, append_images=images[1:], duration=3, loop=0, transparency=0, disposal=2)
